## Our features

#### Изменили функцию, которая дается по умолчанию. Добавили обработку следующих полей:
- operation_type_group
- income_flag
- days_before

Первые две категория представляют собой группы из 3-4 уникальных видов. Создаем по 2-4 столбца.
В `days_before` мы брали минимальное, среднее и максимальное значение


In [ ]:
import pandas as pd


def num_unique_values(x):
    return len(set(x))

def num_values1(x):
    if 1 in x.value_counts():
        return x.value_counts()[1]
    return 0
def num_values2(x):
    if 2 in x.value_counts():
        return x.value_counts()[2]
    return 0
def num_values3(x):
    if 3 in x.value_counts():
        return x.value_counts()[3]
    return 0
def num_values4(x):
    if 4 in x.value_counts():
        return x.value_counts()[4]
    return 0

agg_features = {
        "transaction_number": "max",
        "amnt": ["min", "max", "mean", "median"],
        "currency": num_unique_values,
        'operation_type_group': [num_values1, num_values2, num_values3, num_values4],
        'income_flag' : [num_values1, num_values2],
        'days_before': ['min', 'mean', 'max']

}

In [ ]:
def get_new_features(dataset, target):
    features = dataset.groupby("app_id", as_index=False).agg(agg_features)
    features.columns = ['_'.join(col).strip('_') for col in features.columns.values]
    features = features.join(target.set_index("app_id"), "app_id")
    return features.drop(columns=["app_id", "flag"]), features[["app_id", "flag"]]

#### Для того, чтобы получить features на тестовой выборке(contest) используем функцию `get_new_features_without_flag`

In [ ]:
def get_new_features_without_flag(dataset, target):
    features = dataset.groupby("app_id", as_index=False).agg(agg_features)
    features.columns = ['_'.join(col).strip('_') for col in features.columns.values]
    features = features.join(target.set_index("app_id"), "app_id")
    return features.drop(columns=["app_id"]), features[["app_id"]]

#### Сохранение features
Так как процесс создания занимает относительное долгое время(5-7 минут), мы сохранили текущие датасеты в csv файлы
В будущем планируется также это проделывать

In [ ]:
X_train, y_train = get_features(dataset_train, target_train)

X_train.to_csv("/Users/vladbax6/Codding/code_works/Python/Works/Alpha_ml/Credit score classification/ourfeatures/x_train.csv")
y_train.to_csv("/Users/vladbax6/Codding/code_works/Python/Works/Alpha_ml/Credit score classification/ourfeatures/y_train.csv")

#### Открытие датасеты с features

In [ ]:
X_train = pd.read_csv("/Users/vladbax6/Codding/code_works/Python/Works/Alpha_ml/Credit score classification/ourfeatures/x_train.csv")
y_train = pd.read_csv("/Users/vladbax6/Codding/code_works/Python/Works/Alpha_ml/Credit score classification/ourfeatures/y_train.csv")

## Our models

Для получения текущего результатат используются две разные модели

RandomForestClassifier() by sklearn
CatBoostRegressor() by catboost

Первая получает настроенные аргументы в ручную. Вторая использует `grid_search` для получения лучших аргументов.
Обе модели использую для обучения тестовую выборку, полученную функцией `train_test_split` by sklearn
<br><br>
#### RandomForestClassifier()

In [ ]:
model = RandomForestClassifier(n_estimators=200,
                               bootstrap = True,
                               max_features = 'sqrt' ,max_depth=10,min_samples_split=5, random_state=1,criterion="log_loss" )
model.fit(X_train, y_train["flag"])

y_test_pred = model.predict_proba(X_test)[:, 1]
y_test["flag"] = y_test_pred
y_test.to_csv("result.csv", index=False)

#### CatBoostRegression

In [ ]:
model2 = CatBoostRegressor(loss_function="RMSE")
grid = {'iterations': [190, 180],
        'learning_rate': [0.03, 0.1],
        'depth': [4, 5, 6,],
        'l2_leaf_reg': [0.2, 1, 3]}
train_dataset = Pool(X_train, y_train['flag'])
model2.grid_search(grid, train_dataset)

pred = model.predict(X_test)
y_test['flag'] = pred
y_test.to_csv("other_result.csv", index=False)

##### X_test и y_test получаются с использованием функции `get_new_features_without_flag`
<br>

#### Задачи:
- Поискать новые features
- Использовать GridSearch с RandomForestClassifier()
- Попробовать использовать иную модель
- Для получения итогового результата использовать предсказания обоих моделей, а после выбирать среднее или лучшее среди результатов обоих